<a href="https://colab.research.google.com/github/armandossrecife/piloto/blob/main/analise_designite_gson.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Checa ambiente

In [2]:
!java --version

openjdk 11.0.19 2023-04-18
OpenJDK Runtime Environment (build 11.0.19+7-post-Ubuntu-0ubuntu120.04.1)
OpenJDK 64-Bit Server VM (build 11.0.19+7-post-Ubuntu-0ubuntu120.04.1, mixed mode, sharing)


In [3]:
!python3 --version

Python 3.10.12


In [4]:
!sudo apt install tree

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following NEW packages will be installed:
  tree
0 upgraded, 1 newly installed, 0 to remove and 15 not upgraded.
Need to get 43.0 kB of archives.
After this operation, 115 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu focal/universe amd64 tree amd64 1.8.0-1 [43.0 kB]
Fetched 43.0 kB in 0s (200 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselected package tree.
(Reading database ... 123105 files and directories currently installe

# Funcões

In [9]:
import pandas as pd
import os
import requests

def checa_indice_language(relacao_linguagens):
  indice=-1
  for i, each in enumerate(relacao_linguagens):
    if "Language" in each:
      indice = i
      break
  return indice

def instala_cloc():
  try:
    comando = !sudo apt-get install cloc
    print(comando)
  except Exception as e:
    print(f'Erro: {str(e)}')

def clona_repositorio(url):
  try:
    comando = f'git clone {url}'
    print(f'Clonando o repositório {url}')
    print(comando)
    os.system(comando)
    print('concluido com sucesso!')
  except Exception as e:
    print(f'Erro: {str(e)}')

def get_primeiro_commit(repositorio):
  try:
    os.environ['REPOSITORIO_ORIGINAL'] = repositorio
    # Primeiro commit
    comando = !cd $REPOSITORIO_ORIGINAL && git rev-list --all | tail -n 1
  except Exception as e:
    print(f'Erro: {str(e)}')
  return comando

def get_commit_tag(tag):
  try:
    os.environ['TAG'] = tag
    comando = !cd gson && git rev-list -n 1 $TAG
  except Exception as e:
    print(f'Erro: {str(e)}')
  return comando

def get_ultimo_commit(repositorio):
  try:
    os.environ['REPOSITORIO_ORIGINAL'] = repositorio
    # Primeiro commit
    comando = !cd $REPOSITORIO_ORIGINAL && git rev-list --all | head -n 1
  except Exception as e:
    print(f'Erro: {str(e)}')
  return comando

def cria_pasta_n_commit(repositorio, pasta_n_commit):
  try:
    os.environ['REPOSITORIO_ORIGINAL'] = repositorio
    os.environ['REPOSITORIO_INICIAL'] = pasta_n_commit
    # Cria uma nova pasta para referenciar apenas os dados do 1o commit
    comando = !cp -R $REPOSITORIO_ORIGINAL $REPOSITORIO_INICIAL
  except Exception as e:
    print(f'Erro: {str(e)}')
  return comando

def checkout_n_commit(pasta_n_commit, n_commit):
  try:
    os.environ['REPOSITORIO_INICIAL'] = pasta_n_commit
    os.environ['PRIMEIRO_COMMIT'] = n_commit
    # git checkout $( git rev-list --all | tail -n 1 )
    comando1 = "cd $REPOSITORIO_INICIAL && git checkout $PRIMEIRO_COMMIT"
    os.system(comando1)
    print(f'Checkout do repositorio no commit {n_commit} realizado com sucesso!')
  except Exception as e:
    print(f'Erro: {str(e)}')

def gera_df_info_repositorio(pasta_n_commit):
  try:
    os.environ['REPOSITORIO_INICIAL'] = pasta_n_commit
    # Quantidade de arquivos na pasta nome_repositorio
    qtd_arquivos = !find $REPOSITORIO_INICIAL -type f | wc -l
    # Quantidade de commits do repositorio nome_repositorio
    qtd_commits = !cd $REPOSITORIO_INICIAL && git log --oneline | grep -c .
    # Tamanho (MB) da pasta nome_repositorio
    tamanho_pasta_mb = !du -hs --block-size=1M $REPOSITORIO_INICIAL | cut -f1
    # Quantidade de colaboradores (devs que commitaram pelo menos uma vez no projeto nome_repositorio)
    qtd_colaboradores_commits = !cd $REPOSITORIO_INICIAL && git log --pretty=format:"%ae" | sort -u | grep -v "noreply.github.com" | wc -l
    # Data do primeiro commit do projeto nome_repositorio
    data_primeiro_commit =!cd $REPOSITORIO_INICIAL && git log --reverse --pretty=format:%ad --date=short | head -1
    # Data do ultimo commit do projeto nome_repositorio
    data_ultimo_commit = !cd $REPOSITORIO_INICIAL && git log -1 --pretty=format:%ad --date=short

    dict_info_repo = {
      'nome':pasta_n_commit,
      'qtd_arquivos':qtd_arquivos,
      'qtd_commits':qtd_commits,
      'tamanho_pasta_mb':tamanho_pasta_mb,
      'qtd_colaboradores_commits':qtd_colaboradores_commits,
      'data_primeiro_commit':data_primeiro_commit,
      'data_ultimo_commit':data_ultimo_commit,
    }

    df_info_repo = pd.DataFrame(dict_info_repo)
  except Exception as e:
    print(f'Erro: {str(e)}')
  return df_info_repo

def gera_df_dados(pasta_n_commit):
  lista_linguagens, lista_files, lista_blank, lista_comment, lista_code = list(), list(), list(), list(), list()
  l_temp = []
  vl_temp = []
  os.environ['REPOSITORIO_INICIAL'] = pasta_n_commit
  try:
    # Detalha os tipos de arquivos que aparecem no repositório do nome_repositorio
    relacao_linguagens = !cd $REPOSITORIO_INICIAL && cloc .
    #relacao_linguagens[checa_indice_language(relacao_linguagens):]
    linguagens_encontradas = relacao_linguagens[(checa_indice_language(relacao_linguagens)+2):]
    for linha in linguagens_encontradas:
      elementos = linha.split(' ')
      novos_elementos = []
      linguagem_encontrada = []
      valores_da_linguagem = []
      for item in elementos:
        if item != '':
          novos_elementos.append(item)
      linguagem_encontrada.append(novos_elementos[:-4])
      valores_da_linguagem.append(novos_elementos[-4:])
      l_temp = l_temp + linguagem_encontrada
      vl_temp = vl_temp + valores_da_linguagem

    l_temp = l_temp[:-3]
    vl_temp = vl_temp[:-3]
    for i,each in enumerate(vl_temp):
      lista_files.append(each[0])
      lista_blank.append(each[1])
      lista_comment.append(each[2])
      lista_code.append(each[3])
  except Exception as e:
    print(f'Erro: {str(e)}')
  try:
    dict_dados_linguagens = {
    'linguagens':l_temp,
    'files':lista_files,
    'blank':lista_blank,
    'comment':lista_comment,
    'code':lista_code
    }
    df_dados_linguagens = pd.DataFrame(dict_dados_linguagens)
    df_dados_linguagens['files'] = df_dados_linguagens['files'].apply(lambda x: int(x))
    df_dados_linguagens['blank'] = df_dados_linguagens['blank'].apply(lambda x: int(x))
    df_dados_linguagens['comment'] = df_dados_linguagens['comment'].apply(lambda x: int(x))
    df_dados_linguagens['code'] = df_dados_linguagens['code'].apply(lambda x: int(x))
  except Exception as e2:
    print(f'Erro: {str(e2)}')

  return df_dados_linguagens

def gera_df_dados_repositorios(df_info_repo, df_info_repo_cf):
  df_repo = None
  try:
    l_cf = df_info_repo_cf.columns.tolist()
    c_c1 = df_info_repo.iloc[0].to_list()
    c_cf = df_info_repo_cf.iloc[0].to_list()

    dict_repo= {
        'descricao':l_cf,
        'dados_c1':c_c1,
        'dados_cf':c_cf
    }
    df_repo = pd.DataFrame(dict_repo)
  except Exception as e:
    print(f'Erro: {str(3)}')
  return df_repo

def gera_df_dados_repositorios2(df_info_repo, df_info_repo_cf):
  df_repo = None
  try:
    l_cf = df_info_repo_cf.columns.tolist()
    c_c1 = df_info_repo.iloc[0].to_list()
    c_cf = df_info_repo_cf.iloc[0].to_list()

    dict_repo= {
        'dados_cf':c_cf
    }
    df_repo = pd.DataFrame(dict_repo)
  except Exception as e:
    print(f'Erro: {str(3)}')
  return df_repo

In [10]:
def download_designite():
  try:
    url = 'https://www.designite-tools.com/static/download/DJC/DesigniteJava.jar'  # replace with the URL of the file you want to download
    filename = 'DesigniteJava.jar'  # replace with the desired filename to save the downloaded file
    response = requests.get(url)

    with open(filename, 'wb') as f:
        f.write(response.content)

    print(f'{filename} downloaded successfully!')
  except Excetpion as e:
    print(f'Erro: {str(e)}')

def run_designite(tag, pasta_testes):
  try:
    os.environ['TESTES'] = pasta_testes
    os.environ['GSON'] = tag + 'gson/src/main/java'
    cria_pasta_testes = !mkdir $TESTES
    print(cria_pasta_testes)
    executa_designite = !java -jar DesigniteJava.jar -i $GSON -o $TESTES
    print(executa_designite)
  except Exception as e:
    print(f'Erro: {str(e)}')

def gera_df_design_smells(pasta_testes, lista):
  path_file = pasta_testes + '/' + 'designCodeSmells.csv'
  df_ds = pd.read_csv(path_file)
  elemento = pasta_testes,df_ds
  lista.append(elemento)
  return df_ds

In [13]:
def is_valid_url(url):
    try:
        response = requests.head(url)
        return response.status_code in range(200, 300)  # Check if status code is in the 2xx range
    except requests.exceptions.RequestException:
        return False  # Any error or exception indicates an invalid or non-existent URL

'''
Example usage:
url1 = "https://www.example.com"
url2 = "https://www.invalid-url.com"
print(is_valid_url(url1))  # Output: True
print(is_valid_url(url2))  # Output: False
'''

'\nExample usage:\nurl1 = "https://www.example.com"\nurl2 = "https://www.invalid-url.com"\nprint(is_valid_url(url1))  # Output: True\nprint(is_valid_url(url2))  # Output: False\n'

In [29]:
import re

def get_elements_with_pattern(strings_list, pattern):
    matching_elements = [s for s in strings_list if re.search(pattern, s)]
    return matching_elements

# Example list of strings
strings_list = ['gson-parent-1.0', 'gson-parent-2.0', 'gson-child-1.0', 'gson-parent-3.0']
pattern_to_match = r'gson-parent'

result = get_elements_with_pattern(strings_list, pattern_to_match)
print(result)

['gson-parent-1.0', 'gson-parent-2.0', 'gson-parent-3.0']


# Análise Designite

In [11]:
import requests

url = 'https://www.designite-tools.com/static/download/DJC/DesigniteJava.jar'  # replace with the URL of the file you want to download
filename = 'DesigniteJava.jar'  # replace with the desired filename to save the downloaded file
response = requests.get(url)

with open(filename, 'wb') as f:
    f.write(response.content)

print(f'{filename} downloaded successfully!')

DesigniteJava.jar downloaded successfully!


In [12]:
!git clone https://github.com/google/gson.git

Cloning into 'gson'...
remote: Enumerating objects: 31096, done.
remote: Counting objects: 100% (693/693), done.
remote: Compressing objects: 100% (333/333), done.
remote: Total 31096 (delta 329), reused 556 (delta 235), pack-reused 30403
Receiving objects: 100% (31096/31096), 7.52 MiB | 16.59 MiB/s, done.
Resolving deltas: 100% (15699/15699), done.


In [ ]:
!tree gson

### Pega a lista de tags

In [26]:
lista_tags_gson = !cd gson && git tag

In [30]:
pattern_to_extract = r'gson-parent-'
get_elements_with_pattern(lista_tags_gson, pattern_to_extract)

['gson-parent-2.10',
 'gson-parent-2.10.1',
 'gson-parent-2.4',
 'gson-parent-2.5',
 'gson-parent-2.6',
 'gson-parent-2.6.1',
 'gson-parent-2.6.2',
 'gson-parent-2.7',
 'gson-parent-2.8.0',
 'gson-parent-2.8.1',
 'gson-parent-2.8.2',
 'gson-parent-2.8.3',
 'gson-parent-2.8.4',
 'gson-parent-2.8.5',
 'gson-parent-2.8.6',
 'gson-parent-2.8.7',
 'gson-parent-2.8.8',
 'gson-parent-2.8.9',
 'gson-parent-2.9.0',
 'gson-parent-2.9.1']

# Análise da tag gson-parent-2.4

In [39]:
# Cria uma nova pasta para referenciar apenas os dados do 1o commit
commit_tag_24 = get_commit_tag(tag='gson-parent-2.4')
cria_pasta_n_commit(repositorio='gson', pasta_n_commit='gson_parent_2_4')
checkout_n_commit('gson_parent_2_4', commit_tag_24[0])

Checkout do repositorio no commit 80c13447ed09f774c879b06e40f288e881b75e4e realizado com sucesso!


In [43]:
ls -liath gson_parent_2_4/gson/src/main/java/

total 12K
2228962 drwxr-xr-x 3 root root 4.0K Jul 19 13:12 ./
2228955 drwxr-xr-x 3 root root 4.0K Jul 19 13:12 ../
2228963 drwxr-xr-x 3 root root 4.0K Jul 19 13:12 com/


In [44]:
!java -jar DesigniteJava.jar -i gson_parent_2_4/gson/src/main/java/ -o gson_parent_2_4

Parsing the source code ...
Resolving symbols...
Extracting metrics...
Extracting code smells...
Done.


In [45]:
!ls -liat gson_parent_2_4/designCodeSmells.csv

2228868 -rw-r--r-- 1 root root 5468 Jul 19 13:14 gson_parent_2_4/designCodeSmells.csv


In [60]:
df_design_smells = pd.read_csv('gson_parent_2_4/designCodeSmells.csv')

In [61]:
df_design_smells

,Project Name,Package Name,Type Name,Code Smell
0,java,com.google.gson,JsonNull,Deficient Encapsulation
1,java,com.google.gson,JsonNull,Broken Hierarchy
2,java,com.google.gson,TypeAdapter,Wide Hierarchy
3,java,com.google.gson,TypeAdapter,Cyclic-Dependent Modularization
4,java,com.google.gson,JsonObject,Cyclic-Dependent Modularization
...,...,...,...,...
74,java,com.google.gson.internal.bind,JsonTreeWriter,Cyclic-Dependent Modularization
75,java,com.google.gson.internal.bind,MapTypeAdapterFactory,Unutilized Abstraction
76,java,com.google.gson.internal.bind,Adapter,Unutilized Abstraction
77,java,com.google.gson.internal.bind,ArrayTypeAdapter,Deficient Encapsulation


In [62]:
df_design_smells['Type Name'].unique()

array(['JsonNull', 'TypeAdapter', 'JsonObject', 'TreeTypeAdapter',
       'SingleTypeFactory', 'Gson', 'FutureTypeAdapter', 'JsonElement',
       'JsonIOException', 'TypeAdapterFactory', 'JsonPrimitive',
       'JsonParseException', 'JsonParser', 'JsonDeserializationContext',
       'JsonStreamParser', 'JsonSyntaxException', 'JsonArray',
       'GsonBuilder', 'JsonScope', 'JsonWriter', 'MalformedJsonException',
       'JsonReader', 'Excluder', '$Gson$Types', 'ParameterizedTypeImpl',
       'GenericArrayTypeImpl', 'WildcardTypeImpl', 'Streams',
       'AppendableWriter', 'LinkedHashTreeMap', 'LinkedTreeMapIterator',
       'JsonReaderInternalAccess', 'LinkedTreeMap', 'Node', 'EntrySet',
       'KeySet', 'DateTypeAdapter', 'CollectionTypeAdapterFactory',
       'Adapter', 'TypeAdapterRuntimeTypeWrapper', 'SqlDateTypeAdapter',
       'TypeAdapters', 'EnumTypeAdapter', 'JsonTreeReader',
       'ObjectTypeAdapter', 'TimeTypeAdapter', 'JsonTreeWriter',
       'MapTypeAdapterFactory', 'ArrayT

In [63]:
# Esta lista eh oriunda da analise AMLOC x FOC e das analises dos 3o quartis AMLOC e FOC
# lista_classes_criticas = ['StorageService', 'DatabaseDescriptor', 'ColumnFamilyStore', 'StorageProxy', 'CompactionManager', 'SSTableReader', 'SelectStatement', 'CassandraDaemon', 'SinglePartitionReadCommand', 'NodeProbe', 'MessagingService', 'Config']
lista_classes_criticas = ["JsonReader","JsonWriter","TypeToken","Gson","JsonElement","Streams","ConstructorConstructor","JsonNull","JsonPrimitive","LazilyParsedNumber","Excluder","JsonArray","JsonObject","TypeAdapters","GsonBuilder","LinkedTreeMap"]

In [64]:
pesquisa = df_design_smells['Type Name'].str.contains('|'.join(lista_classes_criticas))

In [65]:
df_critical_classes = df_design_smells[pesquisa]
df_critical_classes

,Project Name,Package Name,Type Name,Code Smell
0,java,com.google.gson,JsonNull,Deficient Encapsulation
1,java,com.google.gson,JsonNull,Broken Hierarchy
4,java,com.google.gson,JsonObject,Cyclic-Dependent Modularization
10,java,com.google.gson,Gson,Cyclic-Dependent Modularization
11,java,com.google.gson,Gson,Insufficient Modularization
13,java,com.google.gson,JsonElement,Cyclic-Dependent Modularization
14,java,com.google.gson,JsonElement,Insufficient Modularization
18,java,com.google.gson,JsonPrimitive,Insufficient Modularization
25,java,com.google.gson,JsonArray,Cyclic-Dependent Modularization
26,java,com.google.gson,JsonArray,Insufficient Modularization


In [66]:
df_critical_classes.columns = ['ProjectName', 'PackageName', 'TypeName', 'CodeSmell']
df_critical_classes = df_critical_classes.query("CodeSmell=='Cyclic-Dependent Modularization' or CodeSmell=='Hub-like Modularization'")
df_critical_classes

,ProjectName,PackageName,TypeName,CodeSmell
4,java,com.google.gson,JsonObject,Cyclic-Dependent Modularization
10,java,com.google.gson,Gson,Cyclic-Dependent Modularization
13,java,com.google.gson,JsonElement,Cyclic-Dependent Modularization
25,java,com.google.gson,JsonArray,Cyclic-Dependent Modularization
34,java,com.google.gson.stream,JsonReader,Cyclic-Dependent Modularization
37,java,com.google.gson.internal,Excluder,Cyclic-Dependent Modularization
42,java,com.google.gson.internal,Streams,Cyclic-Dependent Modularization
47,java,com.google.gson.internal,JsonReaderInternalAccess,Cyclic-Dependent Modularization
62,java,com.google.gson.internal.bind,TypeAdapters,Cyclic-Dependent Modularization


In [67]:
df_analise_designite_critical_classes = df_critical_classes.sort_values(by='TypeName')
df_analise_designite_critical_classes.to_csv('gson_parent_2_4/analise_designite_classes_criticas.csv', index=False)
df_design_smells.to_csv('gson_parent_2_4/analise_designite.csv', index=False)

# Análise da tag 2.10.1

In [56]:
# Cria uma nova pasta para referenciar apenas os dados do 1o commit
commit_tag_2101 = get_commit_tag(tag='gson-parent-2.10.1')
cria_pasta_n_commit(repositorio='gson', pasta_n_commit='gson_parent_2_10_1')
checkout_n_commit('gson_parent_2_10_1', commit_tag_2101[0])

Checkout do repositorio no commit 2ce6a6106c22bf0cce2e39f68b0da2f117d6e66b realizado com sucesso!


In [57]:
!java -jar DesigniteJava.jar -i gson_parent_2_10_1/gson/src/main/java/ -o gson_parent_2_10_1

Parsing the source code ...
Resolving symbols...
Extracting metrics...
Extracting code smells...
Done.


In [68]:
df_design_smells_final = pd.read_csv('gson_parent_2_10_1/designCodeSmells.csv')

In [69]:
df_design_smells_final

,Project Name,Package Name,Type Name,Code Smell
0,java,com.google.gson,JsonNull,Deficient Encapsulation
1,java,com.google.gson,TypeAdapter,Wide Hierarchy
2,java,com.google.gson,TypeAdapter,Cyclic-Dependent Modularization
3,java,com.google.gson,JsonObject,Cyclic-Dependent Modularization
4,java,com.google.gson,JsonObject,Insufficient Modularization
...,...,...,...,...
95,java,com.google.gson.internal.sql,SqlDateTypeAdapter,Cyclic-Dependent Modularization
96,java,com.google.gson.internal.sql,SqlTypesSupport,Deficient Encapsulation
97,java,com.google.gson.internal.sql,SqlTypesSupport,Cyclic-Dependent Modularization
98,java,com.google.gson.internal.sql,SqlTimeTypeAdapter,Cyclic-Dependent Modularization


In [ ]:
pesquisa_final = df_design_smells_final['Type Name'].str.contains('|'.join(lista_classes_criticas))

In [72]:
df_critical_classes_final = df_design_smells_final[pesquisa_final]
df_critical_classes_final.columns = ['ProjectName', 'PackageName', 'TypeName', 'CodeSmell']
df_critical_classes_final = df_critical_classes_final.query("CodeSmell=='Cyclic-Dependent Modularization' or CodeSmell=='Hub-like Modularization'")

In [73]:
df_critical_classes_final

,ProjectName,PackageName,TypeName,CodeSmell
3,java,com.google.gson,JsonObject,Cyclic-Dependent Modularization
6,java,com.google.gson,Gson,Cyclic-Dependent Modularization
8,java,com.google.gson,Gson,Hub-like Modularization
11,java,com.google.gson,JsonElement,Cyclic-Dependent Modularization
22,java,com.google.gson,JsonArray,Cyclic-Dependent Modularization
30,java,com.google.gson.stream,JsonReader,Cyclic-Dependent Modularization
33,java,com.google.gson.internal,Excluder,Cyclic-Dependent Modularization
36,java,com.google.gson.internal,$Gson$Types,Cyclic-Dependent Modularization
41,java,com.google.gson.internal,Streams,Cyclic-Dependent Modularization
45,java,com.google.gson.internal,JsonReaderInternalAccess,Cyclic-Dependent Modularization


In [74]:
df_analise_designite_critical_classes_final = df_critical_classes_final.sort_values(by='TypeName')
df_analise_designite_critical_classes_final.to_csv('gson_parent_2_10_1/analise_designite_classes_criticas.csv', index=False)
df_design_smells_final.to_csv('gson_parent_2_10_1/analise_designite.csv', index=False)